# Correlation-Based Learning Algorithm

## 1. Execute all cells containing functions

In [2]:
import numpy as np
import multiprocessing
from zipfile import BadZipFile
from IPython.display import clear_output

In [ ]:
def gen_omega(n, omega_coefficient):
    omega = np.random.random(n)*omega_coefficient
    return omega

def desired_fea(n_fea, desired_n, cond = 0, multi = 0):
    fea_num = np.zeros(n_fea)
    for i in range(desired_n):
        if cond:
            if multi:
                fea_num[i] = multi
            else:
                fea_num[i] = i + 1
        else:
            fea_num[i] = 1
    return fea_num

def kernel_fn(length, tau_mem, tau_syn, time_ij):
    time = np.arange(0., length, 1.) #ms
    kernel = np.zeros(length)
    eta = tau_mem/tau_syn
    V_norm = eta**(eta/(eta-1))/(eta-1)
    for count in range(length):
        kernel[count] = V_norm*(np.exp(-(time[count]-time_ij)/tau_mem)-np.exp(-(time[count]-time_ij)/tau_syn))
    return kernel

In [ ]:
### Optimized voltage calculation code by Alex ###

def get_memory_len(kernel_array, ratio):
    arr = (kernel_array - ratio*kernel_array.max())[::-1]
    memory_len = len(kernel_array) - np.searchsorted(arr, 0)
    return memory_len

def find_first_spike(V, threshold):
    # Based on the equivalent code to py_find_1st (https://github.com/roebel/py_find_1st)
    ind = np.flatnonzero(V >= threshold)
    if len(ind):
        return ind[0]
    else:
        return -1

def calculate_Vt(unweighted_input, theta, omega):
    spike_time = []
    datalen = unweighted_input.shape[1]
    V = (omega[:,np.newaxis] * unweighted_input).sum(axis=0)
    done = False
    while not done:
        spike_idx = find_first_spike(V, theta)
        if spike_idx == -1:
            done = True
        else:
            spike_time.append(spike_idx)
            mem_len = min(ref_memory_len, datalen - spike_idx)
            V[spike_idx:spike_idx+mem_len] -= theta * ref_kernel[:mem_len]
    return V, spike_time

In [ ]:
def eligibility(data, V_t):
    data = np.concatenate((data, np.zeros((n, 100))), axis=1)
    dic = dict.fromkeys(range(len(data)))
    neuron_eligibility = {value: 0 for value in dic} #To store the eligibility of each neuron
    for i, j in zip(*np.where(data)):
        kernel = np.zeros(data.shape[1])
        mem_len = min(syn_memory_len, data.shape[1] - j)
        kernel[j:j+mem_len] += syn_kernel[:mem_len]
        eligibility = np.sum(np.multiply(V_t, kernel)) #eligibility += individual correlation, dt = 0.001 
        neuron_eligibility[i] += eligibility
    return neuron_eligibility

In [ ]:
def synaptic_update(elig, init_omega, learning_rate, num_spikes, desired_spikes):
    D9 = sorted(elig, key=elig.get, reverse=True)[int(n/10)] #9th decile
    updated_omega = []
    count = 0
    while count < len(elig):
        if num_spikes < desired_spikes:
            if elig[count] > elig[D9]:
                updated_omega = np.append(updated_omega, init_omega[count] + learning_rate)
            else:
                updated_omega = np.append(updated_omega, init_omega[count])
        elif num_spikes > desired_spikes:
            if elig[count] > elig[D9]:
                updated_omega = np.append(updated_omega, init_omega[count] - learning_rate)
            else:
                updated_omega = np.append(updated_omega, init_omega[count])
        else:
            return init_omega
        count += 1
    return updated_omega

In [ ]:
def correlation_training(current_omega, n_cycles, data_sets, fea_idx, learning_rate):
    cur_omega_list = []
    for cycle in range(n_cycles):
        for trial in range(cycle*100, cycle*100+100):
            clear_output(wait=True)
            print("Cycle {}, trial {}.".format(cycle, trial))
            input_data = None
            while input_data is None:
                idx = int(np.random.random()*data_sets)
                try:
                    input_data = np.load("data/data_"+str(idx)+".npz")
                    data, presyn_input, markers, n_fea_occur, fea_time, fea_order = input_data['arr_0'], input_data['arr_1'], input_data['arr_2'], input_data['arr_3'], input_data['arr_4'], input_data['arr_5']
                except (FileNotFoundError, BadZipFile, KeyError):
                    pass
            desired_spikes = sum(n_fea_occur * fea_idx)
            V_t, spike_time = calculate_Vt(presyn_input, theta, current_omega)
            num_spikes = len(spike_time)
            if num_spikes != desired_spikes:
                elig = eligibility(data, V_t)
                current_omega = synaptic_update(elig, current_omega, learning_rate, num_spikes, desired_spikes)
        cur_omega_list.append(current_omega)
    return cur_omega_list

## 2. Training Procedures

### 2.1 Pre-set Parameters

Set the parameters and run the cell below.

Note: Omega_coefficient of 0.022 was chosen based on the average firing rate of approximately 5 Hz for the postsynaptic neuron with initial synaptic efficacies (omega).

In [ ]:
n = 500 # number of input neurons
omega_coefficient = 0.022
theta = 1 # threshold
tau_mem = 20
tau_syn = 5
time_ij = 0
init_kernel_len = 200

# Generate the initial omega for each input neuron
np.random.seed(100000)
omega = gen_omega(n, omega_coefficient)

# Create the PSP kernel, and then remove the negligible tail
syn_kernel = kernel_fn(init_kernel_len, tau_mem, tau_syn, time_ij)
syn_memory_len = get_memory_len(syn_kernel, ratio=0.001)
syn_kernel = syn_kernel[:syn_memory_len]

# Create the refractory kernel, and then remove the negligible tail
ref_kernel = np.exp(- np.arange(1000) / tau_mem)
ref_memory_len = get_memory_len(ref_kernel, ratio=0.001)
ref_kernel = ref_kernel[:ref_memory_len]

### 2.2 Training

The correlation_training function takes 5 arguments, which are current_omega, n_cycles, data_sets, fea_idx, and learning_rate, as inputs and return a list of updated omegas. The updated omega list stores the updated omega after every cycle of training (100 trials each).

a. current_omega: intial omega

b. n_cycles: number of training cycles required

c. data_sets: available number of precomputed data sets in the pool

d. fea_idx: indices of features that the multispike tempotron has to identify

e. learning_rate: the size of update after each error trial

Note: The learning_rate of 0.00001 was chosen according to the original publication (Robert Gütig, 2016).

#### 2.2.1 Identification of one feature with one spike

The desired_fea function is used to generate a list of desired feature numbers.

To generate one spike to identify only one feature, set desired_n to 1 and cond to 0.

In [ ]:
n_cycles = 1000
data_sets = 19999
learning_rate = 0.00001
n_fea = 10 # Total number of features and distractors
desired_n = 1 # Number of desired features

# To generate a list of desired feature indices
fea_idx = desired_fea(n_fea, desired_n, cond = 0, multi = 0)

np.random.seed(int(1e9))
corr_omega1F1S_list = correlation_training(omega, n_cycles, data_sets, fea_idx, learning_rate)
np.save("corr_omega1F1S_list", corr_omega1F1S_list)

#### 2.2.2 Identification of one feature with multiple spikes

To generate multiple spikes to identify one feature, set desired_n to 1, cond to 1, and multi to the desired number of spikes.

In [ ]:
n_cycles = 1000
data_sets = 19999
learning_rate = 0.00001
n_fea = 10 # Total number of features and distractors
desired_n = 1 # Number of desired features

# To generate a list of desired feature indices
fea_idx = desired_fea(n_fea, desired_n, cond = 1, multi = 5) 

np.random.seed(int(1e9))
corr_omega1F5S_list = correlation_training(omega, n_cycles, data_sets, fea_idx, learning_rate)
np.save("corr_omega1F5S_list", corr_omega1F5S_list)

#### 2.2.3 Identification of multiple features with one spike per feature

To generate one spike to identify multiple features, set desired_n to the number of desired features and cond to 0.

In [ ]:
n_cycles = 1000
data_sets = 19999
learning_rate = 0.00001
n_fea = 10 # Total number of features and distractors
desired_n = 5 # Number of desired features

# To generate a list of desired feature indices
fea_idx = desired_fea(n_fea, desired_n, cond = 0, multi = 0) 

np.random.seed(int(1e9))
corr_omega5F1S_list = correlation_training(omega, n_cycles, data_sets, fea_idx, learning_rate)
np.save("corr_omega5F1S_list", corr_omega5F1S_list)

#### 2.2.4 Identification of multiple features with different number of spikes for different features

To generate different number of spikes to identify different features, set desired_n to the number of desired features, cond to 1, and multi to 0.

In [ ]:
n_cycles = 1000
data_sets = 19999
learning_rate = 0.00001
n_fea = 10 # Total number of features and distractors
desired_n = 5 # Number of desired features

# To generate a list of desired feature indices
fea_idx = desired_fea(n_fea, desired_n, cond = 1, multi = 0) 

np.random.seed(int(1e9))
corr_omega5FmS_list = correlation_training(omega, n_cycles, data_sets, fea_idx, learning_rate)
np.save("corr_omega5FmS_list", corr_omega5FmS_list)

#### 2.2.5 Identification of multiple features with a fixed number of spikes (>1) for each feature

To generate a fixed number of spikes that is greater than 1 for multiple features, set desired_n to the number of desired features, cond to 1, and multi to desired number of spikes.

In [ ]:
n_cycles = 1000
data_sets = 19999
learning_rate = 0.00001
n_fea = 10 # Total number of features and distractors
desired_n = 5 # Number of desired features

# To generate a list of desired feature indices
fea_idx = desired_fea(n_fea, desired_n, cond = 1, multi = 5) 

np.random.seed(int(1e9))
corr_omega5F5S_list = correlation_training(omega, n_cycles, data_sets, fea_idx, learning_rate)
np.save("corr_omega5F5S_list", corr_omega5F5S_list)

### 2.3 Multiple runs

In [ ]:
n_cycles = 1000
data_sets = 19999
learning_rate = 0.00001
n_fea = 10 # Total number of features and distractors

fea_idx_list = [desired_fea(n_fea, desired_n=1, cond=0, multi=0),
                desired_fea(n_fea, desired_n=1, cond=1, multi=5),
                desired_fea(n_fea, desired_n=5, cond=0, multi=0),
                desired_fea(n_fea, desired_n=5, cond=1, multi=0),
                desired_fea(n_fea, desired_n=5, cond=1, multi=5)
               ]
filenames = ["corr_omega1F1S_list",
             "corr_omega1F5S_list",
             "corr_omega5F1S_list",
             "corr_omega5FmS_list",
             "corr_omega5F5S_list"
            ]

def run_and_save(filename, fea_idx):
    res = multispike_training(omega, n_cycles, data_sets, fea_idx, learning_rate)
    np.save(filename, res)

with multiprocessing.Pool(5) as pool:
    pool.starmap(run_and_save, zip(filenames, fea_idx_list))